# NFL Dead Money Analysis - Production Visualizations

Generate three key visualizations:
1. **Trend**: Total NFL dead money by year
2. **Heatmap**: Dead money by team and year  
3. **Scatter**: Dead money by player with percentile ranking

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import duckdb

WORKSPACE_ROOT = Path('../')
DBT_PROJECT_DIR = WORKSPACE_ROOT / 'dbt'

# Connect to DuckDB and load data from fact table
conn = duckdb.connect(str(DBT_PROJECT_DIR / 'nfl_dead_money.duckdb'))

# Load player dead money from dbt mart
df_players = conn.execute("""
  SELECT 
    player_name,
    team,
    year,
    dead_cap_millions,
    nfl_percentile,
    pct_of_team_dead_money,
    rank_in_year
  FROM main_marts.fct_dead_money_by_player
""").df()

print(f"Loaded {len(df_players)} records from DuckDB | Teams: {df_players['team'].nunique()} | Years: {df_players['year'].min()}-{df_players['year'].max()}")

Loaded 181 records from DuckDB | Teams: 32 | Years: 2015-2024


In [2]:
# Prepare aggregations
yearly_stats = df_players.groupby('year').agg({
    'dead_cap_millions': ['sum', 'mean', 'count']
}).reset_index()
yearly_stats.columns = ['year', 'total_dead_money', 'avg_dead_money', 'player_count']

team_year_stats = df_players.groupby(['team', 'year'])['dead_cap_millions'].sum().reset_index()
team_year_pivot = team_year_stats.pivot(index='team', columns='year', values='dead_cap_millions')

player_totals = df_players.groupby(['player_name', 'team']).agg({
    'dead_cap_millions': 'sum',
    'year': 'count'
}).reset_index()
player_totals.columns = ['player_name', 'team', 'total_dead_cap', 'years_with_dm']
player_totals['percentile'] = player_totals['total_dead_cap'].rank(pct=True) * 100
player_viz = player_totals[player_totals['total_dead_cap'] > 1.0].copy()

print("✓ Data aggregations complete")

✓ Data aggregations complete


In [3]:
# Visualization 1: Trend
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(
    x=yearly_stats['year'], y=yearly_stats['total_dead_money'],
    mode='lines+markers', name='Total NFL Dead Money',
    line=dict(color='#1f77b4', width=3), marker=dict(size=8)
))
fig_trend.add_trace(go.Scatter(
    x=yearly_stats['year'], y=yearly_stats['avg_dead_money'],
    mode='lines', name='Avg per Player',
    line=dict(color='#ff7f0e', width=2, dash='dash')
))
fig_trend.update_layout(
    title='NFL Total Dead Money Trend (2015-2024)',
    xaxis_title='Year', yaxis_title='Dead Money (Millions $)',
    template='plotly_white', height=500
)
fig_trend.show()
print("✓ Trend chart created")

✓ Trend chart created


In [4]:
# Visualization 2: Heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
    z=team_year_pivot.values, x=team_year_pivot.columns, y=team_year_pivot.index,
    colorscale='YlOrRd',
    colorbar=dict(title='Dead Money (M$)')
))
fig_heatmap.update_layout(
    title='Dead Money by Team & Year',
    xaxis_title='Year', yaxis_title='Team',
    height=700, width=1000
)
fig_heatmap.show()
print("✓ Heatmap created")

✓ Heatmap created


In [5]:
# Visualization 3: Player Scatter
fig_scatter = px.scatter(
    player_viz,
    x='total_dead_cap', y='percentile',
    size='total_dead_cap', color='percentile',
    hover_name='player_name',
    hover_data={'team': True, 'total_dead_cap': ':.2f', 'percentile': ':.1f'},
    color_continuous_scale='Viridis', size_max=50,
    title='NFL Players: Dead Money & Percentile Ranking',
    labels={'total_dead_cap': 'Total Dead Cap (M$)', 'percentile': 'NFL Percentile'},
    height=600, width=1000
)
fig_scatter.update_xaxes(type='log')
fig_scatter.show()
print("✓ Player scatter plot created")

✓ Player scatter plot created


In [6]:
# Data Quality Report
print("\n" + "="*80)
print("DATA QUALITY REPORT")
print("="*80)
print(f"\n✓ Total Records: {len(df_players)}")
print(f"✓ Unique Players: {df_players['player_name'].nunique()}")
print(f"✓ Teams: {df_players['team'].nunique()}/32")
print(f"✓ Year Range: {df_players['year'].min()}-{df_players['year'].max()}")
print(f"✓ Dead Cap: ${df_players['dead_cap_millions'].min():.2f}M - ${df_players['dead_cap_millions'].max():.2f}M")
print(f"✓ Total NFL Dead Money: ${df_players['dead_cap_millions'].sum():.2f}M")
print(f"✓ Null Values: {df_players.isnull().sum().sum()}")
print("\n" + "="*80)


DATA QUALITY REPORT

✓ Total Records: 181
✓ Unique Players: 160
✓ Teams: 32/32
✓ Year Range: 2015-2024
✓ Dead Cap: $1.01M - $41.97M
✓ Total NFL Dead Money: $1338.42M
✓ Null Values: 0



In [7]:
# Export visualizations
output_dir = WORKSPACE_ROOT / 'notebooks' / 'outputs'
output_dir.mkdir(exist_ok=True, parents=True)

fig_trend.write_html(str(output_dir / '01_dead_money_trend.html'))
fig_heatmap.write_html(str(output_dir / '02_team_year_heatmap.html'))
fig_scatter.write_html(str(output_dir / '03_player_scatter.html'))
yearly_stats.to_csv(output_dir / 'data_summary.csv', index=False)

print(f"✓ Visualizations exported to {output_dir}/")
print(f"  - 01_dead_money_trend.html")
print(f"  - 02_team_year_heatmap.html")
print(f"  - 03_player_scatter.html")
print(f"  - data_summary.csv")

✓ Visualizations exported to ../notebooks/outputs/
  - 01_dead_money_trend.html
  - 02_team_year_heatmap.html
  - 03_player_scatter.html
  - data_summary.csv
